In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [4]:
from utils.intermediate_layer_extraction import return_layer_input
from utils.ri_calculator import computeRi,computeRiNaif
import deepmatcher as dm
import pandas as pd

In [6]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.load_state('../../models/amazongoogle_hybrid.pth')

In [7]:
negatives,neg_ids = return_layer_input('../../Structured/Amazon-Google/','test_negatives',128,
                                      hybrid_model,hybrid_model.classifier,device=-1)

In [8]:
positives,pos_ids = return_layer_input('../../Structured/Amazon-Google','test_positives',128,
                                      hybrid_model,hybrid_model.classifier,device=-1)

In [9]:
hybrid_model = hybrid_model.cuda()

## Compare naif approach with gradient approach

### Negatives

In [10]:
def compareMethods(df1,df2,columns):
    best1,best2 = (0,0)
    for i in df1.index:
        for col in columns:
            lval = df1.at[i,col]
            rval = df2.at[i,col]
            if(lval == 0 and rval >0) or(rval <lval and rval >0):
                best2+=1
            elif(rval ==0 and lval >0)or(lval < rval and lval >0):
                best1+=1
    return (best1,best2)

In [11]:
attributes = ['Beer_Name','Brew_Factory_Name','Style','ABV']
attribute_len = int(len(negatives[0][0])/len(attributes))

In [12]:
neg_ri,neg_ri_norms = computeRi(hybrid_model.classifier,attributes,negatives,attribute_len,1)

100%|██████████| 128/128 [00:07<00:00, 16.60it/s]


In [13]:
neg_ri_naif,neg_ri_norms_naif = computeRiNaif(negatives,positives,1,hybrid_model.classifier,attributes,attribute_len)

100%|██████████| 128/128 [00:08<00:00, 15.70it/s]


In [14]:
neg_variation_gradient = pd.DataFrame(data= neg_ri_norms,columns=attributes)
neg_variation_naif = pd.DataFrame(data= neg_ri_norms_naif,columns=attributes)

In [15]:
compareMethods(neg_variation_gradient,neg_variation_naif,attributes)

(7421, 19)

### Positives

In [16]:
pos_ri,pos_ri_norms = computeRi(hybrid_model.classifier,attributes,positives,attribute_len,0)

100%|██████████| 106/106 [00:03<00:00, 32.66it/s]


In [17]:
pos_ri_naif,pos_ri_norms_naif = computeRiNaif(positives,negatives,0,hybrid_model.classifier,attributes,attribute_len)

100%|██████████| 106/106 [01:46<00:00,  1.00s/it]


In [20]:
pos_perturbations_gradient = pd.DataFrame(data= pos_ri_norms,columns=attributes)
pos_perturbations_naif = pd.DataFrame(data= pos_ri_norms_naif,columns=attributes)

In [21]:
compareMethods(pos_perturbations_gradient,pos_perturbations_naif,attributes)

(448, 20)

## Test FP-max

In [1]:
from fpgrowth import fptree

ImportError: cannot import name 'fptree' from 'fpgrowth' (/home/nvidia/workspace/dbgroup/vmartello/deepmatcher-experiments/notebooks/Amazon-Google/fpgrowth.py)